In [ ]:
import json
import re
from tqdm import tqdm

def clean_and_truncate_answer(text: str) -> str:
    """
    답변 텍스트에서 노이즈 패턴을 찾아, 그 패턴이 시작되는 지점부터 뒷부분을 모두 삭제(truncate)하는 함수.
    """
    if not isinstance(text, str):
        return ""

    # 원본 텍스트에서 이중 백슬래시를 단일 백슬래시로 먼저 변환 (일관성 유지)
    cleaned_text = text.replace('\\\\', '\\')

    # 텍스트를 잘라낼 기준이 되는 노이즈 패턴들 (정규표현식)
    # 이 리스트의 순서대로 패턴을 검사합니다.
    truncation_patterns = [
        # 1. '\\n' 또는 유사한 이스케이프 문자가 10번 이상 반복되는 패턴
        r'(\\n){10,}',
        
        # 2. 20자 이상의 긴 구문/문장이 연속으로 반복되는 경우
        r'(.{20,})\1+',
        
        # 3. 2~5자의 대문자 조합이 5번 이상 반복되는 패턴 (예: TXT..., NIST...)
        r'([A-Z]{2,5})\1{4,}'
    ]

    # 가장 먼저 발견되는 노이즈 패턴의 시작 위치를 찾음
    truncate_at = len(cleaned_text) # 기본값은 텍스트 전체 길이
    for pattern in truncation_patterns:
        match = re.search(pattern, cleaned_text)
        if match:
            # 더 먼저 나타나는 패턴을 기준으로 자르기 위해 최소값을 계속 업데이트
            truncate_at = min(truncate_at, match.start())

    # 찾은 위치를 기준으로 텍스트를 자름
    cleaned_text = cleaned_text[:truncate_at]

    # 최종적으로 남은 텍스트에 대해 개행 문자 정리 및 양 끝 공백 제거
    cleaned_text = re.sub(r'\n{3,}', '\n\n', cleaned_text)
    return cleaned_text.strip()

# --- 메인 실행 부분 ---

input_filename = '/workspace/trendyol_cybersec_QA_result.jsonl'
output_filename = 'cybersecurity_data_regex_cleaned_longText.jsonl'

cleaned_lines = 0
total_lines = 0

print(f"'{input_filename}' 파일 정제를 시작합니다...")

try:
    with open(input_filename, 'r', encoding='utf-8') as infile, \
         open(output_filename, 'w', encoding='utf-8') as outfile:
        
        lines = infile.readlines()
        total_lines = len(lines)
        
        for line in tqdm(lines, desc="Regex 정제 처리 중"):
            try:
                data = json.loads(line)
                original_answer = data.get('answer', '')
                
                # 정제 함수를 사용하여 답변을 처리
                cleaned_answer = clean_and_truncate_answer(original_answer)
                
                if len(original_answer) != len(cleaned_answer):
                    cleaned_lines += 1

                data['answer'] = cleaned_answer
                outfile.write(json.dumps(data, ensure_ascii=False) + '\n')

            except json.JSONDecodeError:
                print(f"경고: JSON 파싱 오류가 발생하여 한 줄을 건너뜁니다.")
                continue

    print("\n--- [정제 완료] ---")
    print(f"총 {total_lines}개의 라인을 처리했습니다.")
    print(f"그 중 {cleaned_lines}개의 라인에서 노이즈가 제거(절단)되었습니다.")
    print(f"정제된 데이터가 '{output_filename}' 파일에 저장되었습니다.")

except FileNotFoundError:
    print(f"오류: 입력 파일 '{input_filename}'을 찾을 수 없습니다.")

'/workspace/trendyol_cybersec_QA_result.jsonl' 파일 정제를 시작합니다...


Regex 정제 처리 중: 100%|██████████| 53193/53193 [06:50<00:00, 129.60it/s]


--- [정제 완료] ---
총 53193개의 라인을 처리했습니다.
그 중 32973개의 라인에서 노이즈가 제거(절단)되었습니다.
정제된 데이터가 'cybersecurity_data_regex_cleaned_longText.jsonl' 파일에 저장되었습니다.


: 